In [2]:
import requests
import pandas as pd
import numpy as np
import json
import time
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

In [7]:

def fetch_film_data(title_code):
    #IMDB api meta-data
    url = "https://imdb8.p.rapidapi.com/title/get-meta-data"
    querystring = {"region":"US","ids":title_code}
    headers = {
        'x-rapidapi-host': "imdb8.p.rapidapi.com",
        'x-rapidapi-key': "cb8ae0789fmshd5f578d8c964ca6p18b81fjsn3f8341b9ef41"
        }
    response = requests.request("GET", url, headers=headers, params=querystring)
    meta_json = json.loads(response.text)
    time.sleep(1)
    
    #IMDB api top-billed
    url = "https://imdb8.p.rapidapi.com/title/get-top-cast"
    querystring = {"tconst":title_code}
    headers = {
        'x-rapidapi-host': "imdb8.p.rapidapi.com",
        'x-rapidapi-key': "cb8ae0789fmshd5f578d8c964ca6p18b81fjsn3f8341b9ef41"
        }
    response = requests.request("GET", url, headers=headers, params=querystring)
    actor_json = json.loads(response.text)
    time.sleep(1)
    
    #IMDB api top-crew
    url = "https://imdb8.p.rapidapi.com/title/get-top-crew"
    querystring = {"tconst":title_code}
    headers = {
        'x-rapidapi-host': "imdb8.p.rapidapi.com",
        'x-rapidapi-key': "cb8ae0789fmshd5f578d8c964ca6p18b81fjsn3f8341b9ef41"
        }
    response = requests.request("GET", url, headers=headers, params=querystring)
    crew_json = json.loads(response.text)
    time.sleep(1)
    
    #scrape IMDB PRO
    url = 'https://pro.imdb.com/title/'+title_code+'/details'
    headers = {
        'Cookie':'''session-id=138-1311213-4138762; adblk=adblk_no; ubid-main=131-2177923-5330445; session-id-time=2218844129l; x-main="IS86HKXSJMigI0RnIigNqWd@Pu8ZQWgbXHTCS5NC44TGbsoydTDB4J3y0KW4qQJw"; at-main=Atza|IwEBILUkej479bl-hN19LhAyANxzITDWvKeBnZnnNNWk10j0SijTTrcP1xHYude5_tVJrzPsq30ehR12BnByawJoVpMhpgQ_hNpe0PYjxNHJTAQIAjOTF1wpOsTVFURmn8zA9cCV43yBuMIWlH9cQeyn7lV7Kdd47Hn--VPkHz_W88D4CZ-L1N-H-c-y65FUxELVH6R6EPnh830ssfLXG5L864QRO9hBDwKchx_d6hepwITlwuO8qetimWYabwo8Hty6QX5nZuKY2HNpCWmsakxByc50NIAp7Uy7_bZkvZW2sN8dK9kON4xPKIygVHhMWJQ6wh2-d0DykEh4BKAnSs8dTOaGBf2Wq33d5V4XKmz9RPhyup41DXR5DFVPQojnJoMGVNVo1hd_Xan63I94uYk1JEMN; sess-at-main="p9whlqJpRA5/kki0a+qwvIksIhZyceM2oTm2NUGUstA="; uu=BCYoQbsIiwfBoS5Z-9VBEM4GQeZlJd-x0dsY76F-NJiyuSyqzmSEMKbVSar_-kScSNRzLG9PNA9I%0D%0Ajdjov4ijeWrg0KYq7RquhDNrANLf2r_CKRWG40LVJlIVEtUtjfN7vlSPexThCyrlRCA6MaZXDAXU%0D%0APJiwN-Oy2Uileof_lGTBYNE%0D%0A; pa=BCYo5O3AftOw_UMyDZBOrkb5nNPu-MMicHEKRBry99NXBnJRqeW3prdQxa0PDs0tVA8QWusbH7tZ%0D%0Anraln_N9jpUhsHsZ_jxhsmPTLpuksiUytkmvL8ZfDM5RxRZzJUpLkDX6%0D%0A; session-token="FGBlsQXWFfNa1aXpisRWiZ24Zh7zfYgzF86EWr2infSQ/A6LyfZB++iPHpTBeYr2C2ZzH/C96D7YqEVAjt91KwKNDfhgKCBhgo3cwIzkZ9A2ZqntW/EE9MsAa8/KfK7XLvtj4lUy/nEmiyLVq9Gc2U6Exc1kovQN4H5GmkruH/ywN/ZwSmxBk1VEIqNKeSVbDDnA8JE2OJXuKQNPFsmu5kLk62fEK2cMJMb1pQZw6rU="; csm-hit=tb:FVQ9NP1114J2DT3PZ86C+s-WXASZQ2TS1F9RJ5CAPX5|1588207251592&t:1588207251592&adb:adblk_no"'''
    }
    resp = requests.request("GET", url, headers=headers) 
    soup = BeautifulSoup(resp.text, "html.parser")
    div = soup.find("div", {"id": "box_office_summary"})
    bo = None
    if not div== None:
        bo = div.find_all("div",{"class":"a-column a-span5 a-text-right a-span-last"})
    div1 = soup.find("div", {"id": "contacts"})
    b = None
    if not div1== None:
        b = div1.find_all("span",{"class":"aok-align-center"})
    production = []
    if not b==None:
        for i in b:
            production.append(i.text)
    time.sleep(1)
    
    
    #pull that data together
    title = meta_json[title_code]['title']['title']
    if 'runningTimeInMinutes' in meta_json[title_code]:
        runtime = meta_json[title_code]['title']['runningTimeInMinutes']
    else:
        runtime = 0
    release_date = meta_json[title_code]['releaseDate']
    rating = meta_json[title_code]['certificate']
    if 'metaScore' in meta_json[title_code]:
        metaScore = meta_json[title_code]['metacritic']['metaScore']
    else:
        metaScore = 0
    if 'userScore' in meta_json[title_code]:
        metaUserScore = meta_json[title_code]['metacritic']['userScore']
    else:
        metaUserScore = 0
    if 'rating' in meta_json[title_code]:
        imdbRating = meta_json[title_code]['ratings']['rating']
    else:
        imdbRating = 0
    genre = []
    #print(meta_json[title_code]['genres'])
    #print(type(meta_json[title_code]['genres']))
    if isinstance(meta_json[title_code], list):
        for g in meta_json[title_code]['genres']:
            genre.append(g)
            
    #principal actors        
    actor1 = None 
    actor2 = None
    actor3 = None
    actor4 = None
    actor5 = None
    actor6 = None
    actor7 = None
    actor8 = None
    actor9 = None
    actor10= None
    if not actor_json == None:
        if isinstance(actor_json, list):
            if len(actor_json) >= 1:
                actor1 = actor_json[0]
            if len(actor_json) >= 2:
                actor2 = actor_json[1]
            if len(actor_json) >= 3:
                actor3 = actor_json[2]
            if len(actor_json) >= 4:
                actor4 = actor_json[3]
            if len(actor_json) >= 5:
                actor5 = actor_json[4]
            if len(actor_json) >= 6:
                actor6 = actor_json[5]
            if len(actor_json) >= 7:
                actor7 = actor_json[6]
            if len(actor_json) >= 8:
                actor8 = actor_json[7]
            if len(actor_json) >= 9:
                actor9 = actor_json[8]
            if len(actor_json) >= 10:
                actor10 = actor_json[9]
    
    #pricipal crew
    directors = [] 
    for d in crew_json['directors']:
        directors.append(d['name'])
    writers = []
    for w in crew_json['writers']:
        writers.append(w['name'])
        
    budget = None
    opening_wknd = None
    gross_dom = None
    gross_int = None
    
    if not bo == None:
        if len(bo) >= 1:
            budget = (bo[0].text.replace(',','').replace('\n', '').replace('$', ''))
        if len(bo) >= 2:
            opening_wknd = (bo[1].text.replace(',','').replace('\n', '').replace('$', ''))
        if len(bo) >=3:
            gross_dom = (bo[2].text.replace(',','').replace('\n', '').replace('$', ''))
        if len(bo) >= 4:
            gross_int = (bo[3].text.replace(',','').replace('\n', '').replace('$', ''))
    
    #construct the dataframe
    cols = ['title_code','title', 'runtime', 'release_date',  'rating', 'prod_co','metaScore', 'metaUserScore',
       'imdb_rating', 'genre', 'actor1', 'actor2', 'actor3', 'actor4', 'actor5', 'actor6', 'actor7', 'actor8', 'actor9', 'actor10',
       'directors', 'writers', 'budget', 'opening_wknd', 'gross_dom', 'gross_int']
    df = pd.DataFrame(columns=cols)
    df.loc[0] = [title_code, title, runtime, release_date, rating, production, metaScore, metaUserScore, imdbRating, genre, actor1, actor2,
                actor3, actor4, actor5, actor6, actor7, actor8, actor9, actor10, directors, writers, budget,
                opening_wknd, gross_dom, gross_int]
    
    #return the DF
    return df


In [10]:
sample_df = fetch_film_data('tt8498090')

In [11]:
sample_df.head()

,title_code,title,runtime,release_date,rating,prod_co,metaScore,metaUserScore,imdb_rating,genre,...,actor7,actor8,actor9,actor10,directors,writers,budget,opening_wknd,gross_dom,gross_int
0,tt8498090,Marby Jets Are Go,0,None,None,[Liberty Films],0,0,0,[],...,None,None,None,None,"[Kelly Curtis, John P. Marsh]",[],50000,None,None,None


In [111]:
sample_df.to_csv('data/IMDB_mine_data.csv')
sample_df.head()

,title_code,title,runtime,release_date,rating,prod_co,metaScore,metaUserScore,imdb_rating,genre,...,actor7,actor8,actor9,actor10,directors,writers,budget,opening_wknd,gross_dom,gross_int
0,tt0079302,Home Movies,0,1980-05-16,PG,"[SLC, United Artists Classics]",0,0,0,[],...,/name/nm0876808/,/name/nm0353614/,/name/nm0757854/,/name/nm0096431/,[Brian De Palma],"[Kim Ambler, Brian De Palma, Dana Edelman, Rob...",400000,89134,89134,None


In [21]:
title_file = pd.read_csv('data/us2019_masterFilmList.csv')
a=title_file['titleId'].values
print(a)
type(a)

['tt0276132' 'tt0279481' 'tt0385887' ... 'tt9914522' 'tt9914582'
 'tt9915436']


numpy.ndarray

In [23]:
url = 'https://pro.imdb.com/title/tt7286456/details'
headers = {
    'Cookie':'''session-id=138-1311213-4138762; adblk=adblk_no; ubid-main=131-2177923-5330445; session-id-time=2218844129l; x-main="IS86HKXSJMigI0RnIigNqWd@Pu8ZQWgbXHTCS5NC44TGbsoydTDB4J3y0KW4qQJw"; at-main=Atza|IwEBILUkej479bl-hN19LhAyANxzITDWvKeBnZnnNNWk10j0SijTTrcP1xHYude5_tVJrzPsq30ehR12BnByawJoVpMhpgQ_hNpe0PYjxNHJTAQIAjOTF1wpOsTVFURmn8zA9cCV43yBuMIWlH9cQeyn7lV7Kdd47Hn--VPkHz_W88D4CZ-L1N-H-c-y65FUxELVH6R6EPnh830ssfLXG5L864QRO9hBDwKchx_d6hepwITlwuO8qetimWYabwo8Hty6QX5nZuKY2HNpCWmsakxByc50NIAp7Uy7_bZkvZW2sN8dK9kON4xPKIygVHhMWJQ6wh2-d0DykEh4BKAnSs8dTOaGBf2Wq33d5V4XKmz9RPhyup41DXR5DFVPQojnJoMGVNVo1hd_Xan63I94uYk1JEMN; sess-at-main="p9whlqJpRA5/kki0a+qwvIksIhZyceM2oTm2NUGUstA="; uu=BCYoQbsIiwfBoS5Z-9VBEM4GQeZlJd-x0dsY76F-NJiyuSyqzmSEMKbVSar_-kScSNRzLG9PNA9I%0D%0Ajdjov4ijeWrg0KYq7RquhDNrANLf2r_CKRWG40LVJlIVEtUtjfN7vlSPexThCyrlRCA6MaZXDAXU%0D%0APJiwN-Oy2Uileof_lGTBYNE%0D%0A; pa=BCYo5O3AftOw_UMyDZBOrkb5nNPu-MMicHEKRBry99NXBnJRqeW3prdQxa0PDs0tVA8QWusbH7tZ%0D%0Anraln_N9jpUhsHsZ_jxhsmPTLpuksiUytkmvL8ZfDM5RxRZzJUpLkDX6%0D%0A; session-token="FGBlsQXWFfNa1aXpisRWiZ24Zh7zfYgzF86EWr2infSQ/A6LyfZB++iPHpTBeYr2C2ZzH/C96D7YqEVAjt91KwKNDfhgKCBhgo3cwIzkZ9A2ZqntW/EE9MsAa8/KfK7XLvtj4lUy/nEmiyLVq9Gc2U6Exc1kovQN4H5GmkruH/ywN/ZwSmxBk1VEIqNKeSVbDDnA8JE2OJXuKQNPFsmu5kLk62fEK2cMJMb1pQZw6rU="; csm-hit=tb:FVQ9NP1114J2DT3PZ86C+s-WXASZQ2TS1F9RJ5CAPX5|1588207251592&t:1588207251592&adb:adblk_no"'''
}
resp = requests.request("GET", url, headers=headers)

In [27]:
soup = BeautifulSoup(resp.text, "html.parser")
div = soup.find("div", {"id": "box_office_summary"})
bo = div.find_all("div",{"class":"a-column a-span5 a-text-right a-span-last"})

In [28]:
print(bo)

[<div class="a-column a-span5 a-text-right a-span-last"> $55,000,000
</div>, <div class="a-column a-span5 a-text-right a-span-last"> $96,202,337
</div>, <div class="a-column a-span5 a-text-right a-span-last"> $335,451,311
</div>, <div class="a-column a-span5 a-text-right a-span-last"> $1,074,251,311
</div>]


In [79]:
m_lis = pd.read_csv('data/us2019_masterFilmList.csv')
m_lis = m_lis[m_lis['titleId'] != ]

SyntaxError: invalid syntax (<ipython-input-79-7e38cfe3cb75>, line 2)

In [ ]:
curl 'https://pro.imdb.com/title/tt0079302' -H 'Connection: keep-alive' -H 'Upgrade-Insecure-Requests: 1' -H 'User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36' -H 'Sec-Fetch-Dest: document' -H 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9' -H 'Sec-Fetch-Site: none' -H 'Sec-Fetch-Mode: navigate' -H 'Sec-Fetch-User: ?1' -H 'Accept-Language: en-US,en;q=0.9' -H 'Cookie: session-id=138-1311213-4138762; adblk=adblk_no; ubid-main=131-2177923-5330445; x-main="IS86HKXSJMigI0RnIigNqWd@Pu8ZQWgbXHTCS5NC44TGbsoydTDB4J3y0KW4qQJw"; at-main=Atza|IwEBILUkej479bl-hN19LhAyANxzITDWvKeBnZnnNNWk10j0SijTTrcP1xHYude5_tVJrzPsq30ehR12BnByawJoVpMhpgQ_hNpe0PYjxNHJTAQIAjOTF1wpOsTVFURmn8zA9cCV43yBuMIWlH9cQeyn7lV7Kdd47Hn--VPkHz_W88D4CZ-L1N-H-c-y65FUxELVH6R6EPnh830ssfLXG5L864QRO9hBDwKchx_d6hepwITlwuO8qetimWYabwo8Hty6QX5nZuKY2HNpCWmsakxByc50NIAp7Uy7_bZkvZW2sN8dK9kON4xPKIygVHhMWJQ6wh2-d0DykEh4BKAnSs8dTOaGBf2Wq33d5V4XKmz9RPhyup41DXR5DFVPQojnJoMGVNVo1hd_Xan63I94uYk1JEMN; sess-at-main="p9whlqJpRA5/kki0a+qwvIksIhZyceM2oTm2NUGUstA="; uu=BCYoQbsIiwfBoS5Z-9VBEM4GQeZlJd-x0dsY76F-NJiyuSyqzmSEMKbVSar_-kScSNRzLG9PNA9I%0D%0Ajdjov4ijeWrg0KYq7RquhDNrANLf2r_CKRWG40LVJlIVEtUtjfN7vlSPexThCyrlRCA6MaZXDAXU%0D%0APJiwN-Oy2Uileof_lGTBYNE%0D%0A; pa=BCYo5O3AftOw_UMyDZBOrkb5nNPu-MMicHEKRBry99NXBnJRqeW3prdQxa0PDs0tVA8QWusbH7tZ%0D%0Anraln_N9jpUhsHsZ_jxhsmPTLpuksiUytkmvL8ZfDM5RxRZzJUpLkDX6%0D%0A; session-id-time=2082787201l; session-token="Ew6nYregfPDE/uiLegaQ1kTofa6PMXEai2nBFgjAfw8YRgxkVfb6Hp0uFRFR5dUTZgJzaxdyEzNkoN3rOumzAfW8yAR0E3kr8WXd6lsuVvVtllaAfSWTUKXPtdALoHSNEMLX9uthvsP4iRT2fAvTTRTtzEkyp9RAGSnHznhHx5Q+gcXADyhRDknmTnLC9K3z/l5em9lJAyjiu12z0mk/96IKYDDY1nLjeuiFakc3gYk="; csm-hit=tb:FVQ9NP1114J2DT3PZ86C+s-XNSMRQST9XXN81XJM23W|1588308385538&t:1588308385538&adb:adblk_no' --compressed